In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

print("All libraries imported successfully!")


All libraries imported successfully!


In [2]:
import pandas as pd

# Load the datasets
fake_df = pd.read_csv("Fake.csv")
true_df = pd.read_csv("True.csv")

# Display the first few rows of each dataset
print("Fake News Sample:")
print(fake_df.head())

print("\nTrue News Sample:")
print(true_df.head())


FileNotFoundError: [Errno 2] No such file or directory: 'Fake.csv'

In [3]:
import os
print(os.getcwd())  # This will print the current working directory


C:\Users\DELL\OneDrive\Desktop\Fake_News_Detection


In [4]:
import zipfile

# Extract Fake.csv.zip
with zipfile.ZipFile("Fake.csv.zip", 'r') as zip_ref:
    zip_ref.extractall()

# Extract True.csv.zip
with zipfile.ZipFile("True.csv.zip", 'r') as zip_ref:
    zip_ref.extractall()

print("Files extracted successfully!")


Files extracted successfully!


In [5]:
import pandas as pd

# Load the datasets
fake_df = pd.read_csv("Fake.csv")
true_df = pd.read_csv("True.csv")

# Display the first few rows
print("Fake News Sample:")
print(fake_df.head())

print("\nTrue News Sample:")
print(true_df.head())


Fake News Sample:
                                               title   
0   Donald Trump Sends Out Embarrassing New Year’...  \
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject   
0  Donald Trump just couldn t wish all Americans ...    News  \
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  

True News Sample:
                                               title   
0  As U.S. budget fight looms, Republicans flip

In [6]:
# Check the first few rows of each dataset
print("Fake News Sample:")
print(fake_df.head(), "\n")

print("True News Sample:")
print(true_df.head(), "\n")

# Check column names
print("Fake News Columns:", fake_df.columns)
print("True News Columns:", true_df.columns)

# Check for missing values
print("\nMissing values in Fake News dataset:")
print(fake_df.isnull().sum())

print("\nMissing values in True News dataset:")
print(true_df.isnull().sum())


Fake News Sample:
                                               title   
0   Donald Trump Sends Out Embarrassing New Year’...  \
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject   
0  Donald Trump just couldn t wish all Americans ...    News  \
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017   

True News Sample:
                                               title   
0  As U.S. budget fight looms, Republicans fli

In [7]:
# Select only 'title' and 'text' columns and add a 'label' column
fake_df['label'] = 1  # Fake news -> 1
true_df['label'] = 0  # True news -> 0

# Combine both datasets
df = pd.concat([fake_df[['title', 'text', 'label']], true_df[['title', 'text', 'label']]], axis=0)

# Shuffle the dataset to mix fake and true news
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display dataset structure
print(df.head())
print("\nDataset Shape:", df.shape)


                                               title   
0  Ben Stein Calls Out 9th Circuit Court: Committ...  \
1  Trump drops Steve Bannon from National Securit...   
2  Puerto Rico expects U.S. to lift Jones Act shi...   
3   OOPS: Trump Just Accidentally Confirmed He Le...   
4  Donald Trump heads for Scotland to reopen a go...   

                                                text  label  
0  21st Century Wire says Ben Stein, reputable pr...      1  
1  WASHINGTON (Reuters) - U.S. President Donald T...      0  
2  (Reuters) - Puerto Rico Governor Ricardo Rosse...      0  
3  On Monday, Donald Trump once again embarrassed...      1  
4  GLASGOW, Scotland (Reuters) - Most U.S. presid...      0  

Dataset Shape: (44898, 3)


In [8]:
# Select only 'title' and 'text' columns and add a 'label' column
fake_df['label'] = 1  # Fake news -> 1
true_df['label'] = 0  # True news -> 0

# Combine both datasets
df = pd.concat([fake_df[['title', 'text', 'label']], true_df[['title', 'text', 'label']]], axis=0)

# Shuffle the dataset to mix fake and true news
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Display dataset structure
print(df.head())
print("\nDataset Shape:", df.shape)


                                               title   
0  Ben Stein Calls Out 9th Circuit Court: Committ...  \
1  Trump drops Steve Bannon from National Securit...   
2  Puerto Rico expects U.S. to lift Jones Act shi...   
3   OOPS: Trump Just Accidentally Confirmed He Le...   
4  Donald Trump heads for Scotland to reopen a go...   

                                                text  label  
0  21st Century Wire says Ben Stein, reputable pr...      1  
1  WASHINGTON (Reuters) - U.S. President Donald T...      0  
2  (Reuters) - Puerto Rico Governor Ricardo Rosse...      0  
3  On Monday, Donald Trump once again embarrassed...      1  
4  GLASGOW, Scotland (Reuters) - Most U.S. presid...      0  

Dataset Shape: (44898, 3)


In [9]:
from sklearn.model_selection import train_test_split

# Splitting the dataset (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Display dataset sizes
print("Training set size:", len(X_train))
print("Testing set size:", len(X_test))


Training set size: 35918
Testing set size: 8980


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words="english", max_df=0.7)

# Transform the text data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Display shape of transformed data
print("TF-IDF Training Data Shape:", X_train_tfidf.shape)
print("TF-IDF Testing Data Shape:", X_test_tfidf.shape)


TF-IDF Training Data Shape: (35918, 111193)
TF-IDF Testing Data Shape: (8980, 111193)


In [11]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Model training completed
print("Model training completed!")


Model training completed!


In [12]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions on the test data
y_pred = model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

# Display detailed classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Model Accuracy: 0.982739420935412

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.99      0.98      4270
           1       0.99      0.98      0.98      4710

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



In [13]:
def predict_news(news_text):
    # Transform the input text using the same TF-IDF vectorizer
    news_tfidf = tfidf_vectorizer.transform([news_text])

    # Predict using the trained model
    prediction = model.predict(news_tfidf)

    # Output result
    return "Fake News" if prediction[0] == 1 else "Real News"

# Example test
sample_news = "The government has announced a new economic policy to boost growth."
print("Prediction:", predict_news(sample_news))


Prediction: Real News


In [14]:
import joblib

# Save the TF-IDF vectorizer
joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")

# Save the trained model
joblib.dump(model, "fake_news_model.pkl")

print("Model and vectorizer saved successfully!")


Model and vectorizer saved successfully!


In [15]:
# Load the saved TF-IDF vectorizer and model
loaded_vectorizer = joblib.load("tfidf_vectorizer.pkl")
loaded_model = joblib.load("fake_news_model.pkl")

# Function to make predictions
def predict_news_from_saved_model(news_text):
    news_tfidf = loaded_vectorizer.transform([news_text])
    prediction = loaded_model.predict(news_tfidf)
    return "Fake News" if prediction[0] == 1 else "Real News"

# Example test
sample_news = "A new scientific breakthrough has been made in the fight against climate change."
print("Prediction:", predict_news_from_saved_model(sample_news))


Prediction: Fake News


In [1]:
import pandas as pd

df_fake = pd.read_csv("Fake.csv")
df_real = pd.read_csv("True.csv")

print("Fake News Count:", len(df_fake))
print("Real News Count:", len(df_real))


Fake News Count: 23481
Real News Count: 21417
